In [1]:
import torch

from dpp_nets.my_torch.simulator import SimFilter

In [2]:
network_params = {'set_size': 40, 'n_clusters': 10, 'max_sig': 10}
dtype = torch.DoubleTensor

In [3]:
torch.manual_seed(10)
a = SimFilter(network_params, dtype)

In [4]:
torch.manual_seed(14)
a.train(1000, 10, 1e-5, 4)

Loss at it  100  is:  195.05
Loss at it  200  is:  138.95
Loss at it  300  is:  100.7
Loss at it  400  is:  103.3
Loss at it  500  is:  100.025
Loss at it  600  is:  94.875
Loss at it  700  is:  85.5
Loss at it  800  is:  65.475
Loss at it  900  is:  77.925
Loss at it  1000  is:  75.275


In [ ]:
torch.manual_seed(10)
a = SimFilter(network_params, dtype)

In [ ]:
torch.manual_seed(14)
a.train(1000, 10, 1e-5, 4)

In [3]:
from dpp_nets.my_torch.simulator import SimKDPPDeepSet
network_params = {'set_size': 40, 'n_clusters': 10}
dtype = torch.DoubleTensor

In [4]:
sim = SimKDPPDeepSet(network_params, dtype)

In [ ]:
train_it = 10000
batch_size = 10
lr = 1e-5
alpha_iter = 4

sim.train(train_it, batch_size, lr, alpha_iter)

Loss at it  100  is:  435.2065574085594
Loss at it  200  is:  391.55441993019105
Loss at it  300  is:  318.53462535113493
Loss at it  400  is:  317.9016917079392
Loss at it  500  is:  303.5734380543932
Loss at it  600  is:  280.0291490456058
Loss at it  700  is:  283.301057388817
Loss at it  800  is:  261.0169772292295
Loss at it  900  is:  248.7966638255713
Loss at it  1000  is:  248.59262123431193
Loss at it  1100  is:  260.46403124043985
Loss at it  1200  is:  251.11081306481452
Loss at it  1300  is:  248.45816139572958
Loss at it  1400  is:  243.99184997454648
Loss at it  1500  is:  244.03976213840158
Loss at it  1600  is:  254.72442688279645
Loss at it  1700  is:  238.3406325928701
Loss at it  1800  is:  236.23823202083952
Loss at it  1900  is:  230.22930301265103
